In [1]:
from load_data import read_data, read_env
import os

In [ ]:
# API key
read_env(os.getcwd()[:-3])

In [5]:
read_data(os.getcwd()[:-3] + "/data")

,item_1_short,Labelling
0,ITEM 1. BUSINESS ﻿ INTRODUCTION ﻿ Evolving Sys...,Evolving Systems provides real-time digital en...
1,"ITEM 1. BUSINESS Our Business We develop, mark...","We develop, market, license, host, and support..."
2,ITEM 1. Business Overview Model N is a leading...,Model N is a leading provider of cloud revenue...
3,Item 1. Business Overview Datadog is the monit...,Datadog is the monitoring and analytics platfo...
4,Item 1. Business Overview AudioEye is an indus...,AudioEye is an industry-leading software solut...
...,...,...
6,Item 1. Business Overview Our mission is to ma...,As a leading provider of cloud-based software ...
7,"ITEM 1.BUSINESS Overview Ideanomics, Inc. (“Id...","Ideanomics, Inc. (“Ideanomics” or the “Company..."
8,"ITEM 1. BUSINESS Solely for convenience, trade...","Overview BIO-key International, Inc. (the “Com..."
9,Item 1. Business Factors History The Company w...,"The Company was formed in Delaware on June 5, ..."
